### Strategy:
1. Adjust Doc Freqency for PreProc
2. Preproc Data
3. Build, param, train, and predict data
4. Document results
5. Repeat

In [1]:
###### GLOBAL ###########

INSTALL_NEEDED = False

## Load CSV Datasets
MAKE_DATA = False
ROWS = 105000 # Limit 105000
DF1, DF2, DF3 = False, True, False
ALL_ROWS = False

# Make CSV for ValData
VAL_DATA = False

# Build PreProccessed Datasets
## PICK ONE FOR PREPROCESSING ##
# If all False, will attempt to load our preproccessed data
PREPROCESS = False # Use our Preproccessing
BERT_PREPROCESS = False # Use BERT Preproceccessing
BERT_LOAD = False # Load Preproccessed BERT Data

# Set model types for session ('SGD', 'XGB', 'ADA')
MODEL_TYPE = 'XGB'

# Build Models
NEW_MODELS = True

# Search for Parameters
PARAMETER_SEARCH = False
OLD_PARAMS = False

# Train Models
TRAIN = True

# Predict Models
PREDICT_TEST = True
FULL_PREDICT_TEST = False

###### GLOBAL ###########

In [2]:
pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement string (from versions: none)
ERROR: No matching distribution found for string
Note: you may need to restart the kernel to use updated packages.


In [3]:
# pip install transformers

In [4]:
# pip install nltk

In [5]:

if INSTALL_NEEDED == True:  
  import nltk
  nltk.download('punkt')
  nltk.download('stopwords')
  nltk.download('wordnet')

In [6]:
import pandas as pd
import pickle

from data.data import Data as data_csv
from scripts.Preprocessing_full import *
from model.model import *

from sklearn.utils import shuffle



if MAKE_DATA == True:
  data_raw = data_csv().get_all_data()
  
  

  data_list = []

  # 8670 ROWS INPUT #
  if DF1:
    df1 = data_raw['twitter_MBTI']
    df1 = df1[['label', 'text']]
    df1.rename(columns={'label': 'type'}, inplace=True)
    if ALL_ROWS == False:
      df1 = df1.sample(n = ROWS, ignore_index = True)
    data_list.append(df1)


  # 106062 ROWS INPUT #
  if DF2:
    df2 = data_raw['MBTI 500']
    df2 = df2[['type', 'posts']]
    df2.rename(columns={'posts': 'text'}, inplace=True)
    df2 = df2.sample(frac=1).reset_index(drop=True)
    if VAL_DATA:
        dfval = df2.iloc[ROWS:]
        dfval.to_csv('data/csv/valdata.csv')    
        print("Val data created, path = data/csv/valdata.csv")
    if ALL_ROWS == False:
      df2 = df2.iloc[:ROWS]
    data_list.append(df2)


  # 7811 ROWS INPUT #
  if DF3:
    df3 = data_raw['mbti_1']
    df3.rename(columns={'posts': 'text'}, inplace=True)
    if ALL_ROWS == False:
      df3 = df3.sample(n = ROWS, ignore_index = True)
    data_list.append(df3)

  #####################################################



  ## Combine all data 
  data_combined = pd.concat(data_list, ignore_index=True)
  print(f"data_combined ready with shape: {data_combined.shape}")

else:
  print("Data already loaded")


Data already loaded


In [7]:
from scripts.Preprocessing_full import training_preprocessing, training_oversampling, training_balancing, training_vectorize
# from scripts.BERT_preprocessing import bert_training_preprocessing


if PREPROCESS:
  df_pp = training_preprocessing(data_combined)
  df_pp_os = training_oversampling(df_pp)
  df_pp_os_bal = training_balancing(df_pp_os)
  df_list = training_vectorize(df_pp_os_bal)

  df_list[0].dropna(inplace = True)
  df_list[1].dropna(inplace = True)
  df_list[2].dropna(inplace = True)
  df_list[3].dropna(inplace = True)

  with open(f'data/prepro/dflist_pp_{ROWS}.pkl', 'wb') as file:
    pickle.dump(df_list, file)
  
  print(f"df_list_pp_{ROWS} has been saved")
  print(f"IE: {df_list[0].shape}")
  print(f"SN: {df_list[1].shape}")
  print(f"FT: {df_list[2].shape}")
  print(f"JP: {df_list[3].shape}")

elif BERT_PREPROCESS:
#   df_list = bert_training_preprocessing(data_combined)

  df_list[0].dropna(inplace = True)
  df_list[1].dropna(inplace = True)
  df_list[2].dropna(inplace = True)
  df_list[3].dropna(inplace = True)

  with open(f'data/prepro/dflist_pp_BERT_{ROWS}.pkl', 'wb') as file:
    pickle.dump(df_list, file)
  
  print(f"df_list_pp_BERT_{ROWS} has been saved")
  
  with open(f'data/prepro/dflist_pp_BERT_{ROWS}.pkl', 'rb') as file:
        df_list = pickle.load(file)
  print(f"data/prepro/dflist_pp_BERT_{ROWS} has been loaded")
  print(f"IE: {df_list[0].shape}")
  print(f"SN: {df_list[1].shape}")
  print(f"FT: {df_list[2].shape}")
  print(f"JP: {df_list[3].shape}")

elif BERT_LOAD:
  with open(f'data/prepro/dflist_pp_BERT_{ROWS}.pkl', 'rb') as file:
    df_list = pickle.load(file)
  print(f"data/prepro/dflist_pp_BERT_{ROWS} has been loaded as df_list")
  print(f"IE: {df_list[0].shape}")
  print(f"SN: {df_list[1].shape}")
  print(f"FT: {df_list[2].shape}")
  print(f"JP: {df_list[3].shape}")
  
else:
  with open(f'data/prepro/dflist_pp_{ROWS}.pkl', 'rb') as file:
    df_list = pickle.load(file)
  print(f"df_list_pp_{ROWS} has been loaded as df_list")
  print(f"IE: {df_list[0].shape}")
  print(f"SN: {df_list[1].shape}")
  print(f"FT: {df_list[2].shape}")
  print(f"JP: {df_list[3].shape}")

df_list_pp_105000 has been loaded as df_list
IE: (50306, 2380)
SN: (18212, 2376)
FT: (72990, 2351)
JP: (87961, 2373)


<!--  -->

In [8]:
if NEW_MODELS:
  model_list = initialize_models(new_models = True,
                               params = None,
                               model_type = MODEL_TYPE)
  print("model_list is NEW")
else:
  model_list = load_models_pkl()
  print("model_list is OLD")

model_list is NEW


In [9]:
if PARAMETER_SEARCH:
  param_list = grid_search_all_models(df_list, 
                          model_type=MODEL_TYPE, 
                          rows_for_search=1000)
  with open(f'model/param_list_{MODEL_TYPE}.pkl', 'wb') as file:
    pickle.dump(param_list, file)
  
  model_list = initialize_models(params = param_list,
                               model_type=MODEL_TYPE)
  print("model_list is optimized on new params")
  
elif OLD_PARAMS:
  with open(f'model/param_list_{MODEL_TYPE}.pkl', 'rb') as file:
    param_list = pickle.load(file)

  model_list = initialize_models(params = param_list,
                               model_type=MODEL_TYPE) 
  
  print("model_list is optimized on old params")

else:
    print("no params applied to model_list")

no params applied to model_list


In [10]:
if TRAIN:
  history = train_model(df_list, model_list)
else:
  print("Not Training")

F1-score: 73.801%
Model: GradientBoostingClassifier(n_estimators=500)
MBTI Type: IE
F1-score: 76.644%
Model: GradientBoostingClassifier(n_estimators=500)
MBTI Type: NS
F1-score: 83.914%
Model: GradientBoostingClassifier(n_estimators=500)
MBTI Type: TF
F1-score: 71.986%
Model: GradientBoostingClassifier(n_estimators=500)
MBTI Type: JP


In [11]:
from scripts.Preprocessing_full import prediction_preprocessing, prediction_vectorize
from scripts.BERT_preprocessing import bert_prediction_preprocessing


if PREDICT_TEST:
  j = 0
  df = pd.read_csv('data/csv/valdata.csv')
  for i in range(101):
    df = df.sample(n = 100, ignore_index=True)
    estimation = df['type'].tolist()[0]
    df_pred = df.drop(columns=['Unnamed: 0', 'type'])
    if BERT_LOAD or BERT_PREPROCESS:
        df_pred_list = bert_prediction_preprocessing(df_pred)
    else:
        df_pred_pp = prediction_preprocessing(df_pred)
        df_pred_list = prediction_vectorize(df_pred_pp)
    pred = predict_model(texts = df_pred_list, verbose = False)
    print(f"Prediction = {pred['type_prediction']} {pred['type_score']}")
    print(f"Actual = {estimation}")
    if pred['type_prediction'] == estimation:
      j += 1
  print(f"{j} fully correct MBTI types out of {i}")


/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a mode

Prediction = ENFP 0.6952175265761695
Actual = INTJ
Prediction = INTJ 0.7400170943745221
Actual = INTP
Prediction = INTJ 0.7664580939773898
Actual = INTJ
Prediction = INTP 0.6574588195372144
Actual = ENFP
Prediction = ENFP 0.8088982998644572
Actual = ENTP
Prediction = ESTP 0.7176750539134926
Actual = ISTJ
Prediction = INFJ 0.6473689080344409
Actual = INTP
Prediction = INTJ 0.7579992942674685
Actual = INTJ
Prediction = INTJ 0.651340672948082
Actual = INTJ
Prediction = ESFP 0.6007951729906054
Actual = INFJ
Prediction = INFJ 0.8320796245522502
Actual = INFJ
Prediction = ENTJ 0.676302412227994
Actual = INTJ
Prediction = INFP 0.7273434034187714
Actual = INTP
Prediction = ISFJ 0.7151003731299695
Actual = INFP
Prediction = INFJ 0.6966512427189673
Actual = INFJ
Prediction = INTJ 0.7327953919545027
Actual = INTJ
Prediction = INFP 0.6743971605402805
Actual = INFP
Prediction = INFP 0.6075371291126003
Actual = INFP
Prediction = INTP 0.7393619498469479
Actual = INTJ
Prediction = INFJ 0.8320796245522

# DF Comparison (n=7000)
### DF1 ------------------------------------------
F1-score: 52.669999999999995%
MBTI Type: EI

F1-score: 53.102000000000004%
MBTI Type: NS

F1-score: 49.124%
MBTI Type: TF

F1-score: 57.786% 
MBTI Type: PJ

### DF2 ------------------------------------------
F1-score: 60.718%
MBTI Type: EI
 
F1-score: 60.75000000000001%
MBTI Type: NS 

F1-score: 78.408%
MBTI Type: TF

F1-score: 54.04600000000001%
MBTI Type: PJ


### DF 3 -------------------------------------


F1-score: 58.536%
MBTI Type: IE

F1-score: 58.318999999999996%
MBTI Type: NS

F1-score: 66.681%
MBTI Type: TF

F1-score: 57.265%
MBTI Type: JP



# Tuned Model Comparison (n=ALLROWS) MinDoc = .02, MaxDoc = .8
### SGD ------------------------------------------
"53 fully correct MBTI types out of 100"

F1-score: 73.219%
MBTI Type: EI

F1-score: 77.578%
MBTI Type: NS

F1-score: 78.888%
MBTI Type: TF

F1-score: 67.625%
MBTI Type: PJ

### XGB ------------------------------------------
"{j} fully correct MBTI types out of 100"

F1-score: 60.718%
MBTI Type: EI
 
F1-score: 60.75000000000001%
MBTI Type: NS 

F1-score: 78.408%
MBTI Type: TF

F1-score: 54.04600000000001%
MBTI Type: PJ


### ADA ------------------------------------------
"{j} fully correct MBTI types out of 100"

F1-score: 58.536%
MBTI Type: IE

F1-score: 58.318999999999996%
MBTI Type: NS

F1-score: 66.681%
MBTI Type: TF

F1-score: 57.265%
MBTI Type: JP



# Tuned Model Comparison (n=ALLROWS) MinDoc = .02, MaxDoc = .1
### SGD ------------------------------------------
3 fully correct MBTI types out of 100

F1-score: 33.571%
MBTI Type: EI

F1-score: 52.43299999999999%
MBTI Type: NS

F1-score: 71.90100000000001%
MBTI Type: TF

F1-score: 50.18%
MBTI Type: PJ


### ADA ------------------------------------------
8 fully correct MBTI types out of 100

F1-score: 62.705%
MBTI Type: IE

F1-score: 69.761%
MBTI Type: NS

F1-score: 64.129%
MBTI Type: TF

F1-score: 59.803%
MBTI Type: JP



# Tuned Model Comparison (n=ALLROWS) MinDoc = .02, MaxDoc = .4
### SGD ------------------------------------------
"0 fully correct MBTI types out of 100"

F1-score: 73.831%
MBTI Type: EI

F1-score: 77.83%
MBTI Type: NS

F1-score: 78.205%
MBTI Type: TF

F1-score: 66.624%
MBTI Type: PJ

### XGB ------------------------------------------
"{j} fully correct MBTI types out of 100"

F1-score: 60.718%
MBTI Type: EI
 
F1-score: 60.75000000000001%
MBTI Type: NS 

F1-score: 78.408%
MBTI Type: TF

F1-score: 54.04600000000001%
MBTI Type: PJ


### ADA ------------------------------------------
"8 fully correct MBTI types out of 100"

F1-score: 74.789
MBTI Type: IE

F1-score: 78.479%
MBTI Type: NS

F1-score: 78.49199999999999%
MBTI Type: TF

F1-score: 67.766%
MBTI Type: JP



# Tuned Model Comparison (n=ALLROWS) MinDoc = .02, MaxDoc = .9
### SGD ------------------------------------------
0 fully correct MBTI types out of 100

F1-score: 65.09%
MBTI Type: EI

F1-score: 48.423%
MBTI Type: NS

F1-score: 40.599000000000004%
MBTI Type: TF

F1-score: 33.452%
MBTI Type: PJ


### ADA ------------------------------------------
14 fully correct MBTI types out of 100

F1-score: 56.257999999999996%
MBTI Type: IE

F1-score: 70.999%
MBTI Type: NS

F1-score: 68.296%
MBTI Type: TF

F1-score: 60.927%
MBTI Type: JP



# Tuned Model Comparison (n=ALLROWS) MinDoc = .02, MaxDoc = .7
### SGD ------------------------------------------
1 fully correct MBTI types out of 100

F1-score: 52.112%
MBTI Type: EI

F1-score: 69.526%
MBTI Type: NS

F1-score: 59.90299999999999%
MBTI Type: TF

F1-score: 33.452%
MBTI Type: PJ


### ADA ------------------------------------------
2 fully correct MBTI types out of 100

F1-score: 66.07600000000001%
MBTI Type: IE

F1-score: 65.571%
MBTI Type: NS

F1-score: 62.552%
MBTI Type: TF

F1-score: 60.831999999999994%
MBTI Type: JP



# Tuned Model Comparison (n=5000) MinDoc = .02, MaxDoc = .7, DF2 Only
### SGD ------------------------------------------
## 11 sec
13 fully correct MBTI types out of 100

F1-score: 33.396%
MBTI Type: EI

F1-score: 34.605999999999995%
MBTI Type: NS

F1-score: 62.931000000000004%
MBTI Type: TF

F1-score: 33.95%
MBTI Type: PJ

### XGB ------------------------------------------
## 21 min
37 fully correct MBTI types out of 100

F1-score: 63.349999999999994%
MBTI Type: EI
 
F1-score: 62.55%
MBTI Type: NS 

F1-score: 63.526%
MBTI Type: TF

F1-score: 62.999%
MBTI Type: PJ


### ADA ------------------------------------------
## 2 min
19 fully correct MBTI types out of 100

F1-score: 60.58%
MBTI Type: IE

F1-score: 61.394000000000005%
MBTI Type: NS

F1-score: 70.284%
MBTI Type: TF

F1-score: 57.07%
MBTI Type: JP



# Tuned Model Comparison (n=105000) MinDoc = .02, MaxDoc = .7, DF2 Only, Val Data Set
### SGD ------------------------------------------
## 1 min
39 fully correct MBTI types out of 100

F1-score: 66.196%
MBTI Type: EI

F1-score: 61.043000000000006%
MBTI Type: NS

F1-score: 82.717%
MBTI Type: TF

F1-score: 70.094%
MBTI Type: PJ

### XGB ------------------------------------------
## 7 hrs 40 mins
41 fully correct MBTI types out of 100

F1-score: 74.167%
MBTI Type: EI
 
F1-score: 78.706%
MBTI Type: NS 

F1-score: 83.699%
MBTI Type: TF

F1-score: 71.437%
MBTI Type: PJ


### ADA ------------------------------------------
## 43 min
21 fully correct MBTI types out of 100

F1-score: 68.35799999999999%
MBTI Type: IE

F1-score: 66.898%
MBTI Type: NS

F1-score: 74.72999999999999%
MBTI Type: TF

F1-score: 54.647%
MBTI Type: JP



# Predicting and comparing DF1, DF2, and DF3 on best model (XGB 105k)

### XGB ------------------------------------------
## 7 hrs 40 mins

### twitter_MBTI as Val Set (43min)

445 fully correct MBTI types out of 7810

5.69782% correct full types

15423 MBTI letters correct out of 31244 total

49.36308% correct individual types


### MBTI 500 as Val Set (5min)

449 fully correct MBTI types out of 1066

42.12008% correct full types

3412 MBTI letters correct out of 4268 total

79.94377% correct individual types


### mbti_1 as Val Set

1574 fully correct MBTI types out of 8674

18.14618% correct full types

22519 MBTI letters correct out of 34700 total

64.89625% correct individual types


### Model Stats
F1-score: 74.167%
MBTI Type: EI
 
F1-score: 78.706%
MBTI Type: NS 

F1-score: 83.699%
MBTI Type: TF

F1-score: 71.437%
MBTI Type: PJ

In [12]:
if FULL_PREDICT_TEST:

    # Iterables
    j = 0
    l = 0
    n = 0

    # Get Data
    df = pd.read_csv('data/csv/twitter_MBTI.csv')
    df= df[['label', 'text']]
    df.rename(columns={'label': 'type'}, inplace=True)

    for i in range(len(df)):
        df_shuf = df.sample(n = 1, ignore_index=True)
        estimation = df_shuf['type'].tolist()[0].upper()
        df_pred = df_shuf.drop(columns='type')

        if BERT_LOAD or BERT_PREPROCESS:
            df_pred_list = bert_prediction_preprocessing(df_pred)
        else:
            df_pred_pp = prediction_preprocessing(df_pred)
            df_pred_list = prediction_vectorize(df_pred_pp)

        
        pred = predict_model(texts = df_pred_list, verbose = False)
        print(f"Prediction = {pred['type_prediction']} {pred['type_score']}")
        print(f"Actual = {estimation}")
        if pred['type_prediction'] == estimation:
            j += 1
        if pred['type_prediction'][0] == estimation[0]:
            l += 1
        if pred['type_prediction'][1] == estimation[1]:
            l += 1
        if pred['type_prediction'][2] == estimation[2]:
            l += 1
        if pred['type_prediction'][3] == estimation[3]:
            l += 1
        n += 4
    
    print("---------------------------")
    print(f"twitter_MBTI as Val Set\n")
    print(f"{j} fully correct MBTI types out of {i}\n")
    print(f"{round((100*(j/i)),5)}% correct full types\n")
    print(f"{l} MBTI letters correct out of {n} total\n")
    print(f"{round((100*(l/n)),5)}% correct individual types\n")

In [13]:
if FULL_PREDICT_TEST: 
    j = 0
    l = 0
    n = 0

    df = pd.read_csv('data/csv/valdata.csv')
    for i in range(len(df)):
        df_shuf = df.sample(n = 1, ignore_index=True)
        estimation = df_shuf['type'].tolist()[0]
        df_pred = df_shuf.drop(columns=['Unnamed: 0', 'type'])

        if BERT_LOAD or BERT_PREPROCESS:
            df_pred_list = bert_prediction_preprocessing(df_pred)
        else:
            df_pred_pp = prediction_preprocessing(df_pred)
            df_pred_list = prediction_vectorize(df_pred_pp)

        
        pred = predict_model(texts = df_pred_list, verbose = False)
        print(f"Prediction = {pred['type_prediction']} {pred['type_score']}")
        print(f"Actual = {estimation}")
        if pred['type_prediction'] == estimation:
            j += 1
        if pred['type_prediction'][0] == estimation[0]:
            l += 1
        if pred['type_prediction'][1] == estimation[1]:
            l += 1
        if pred['type_prediction'][2] == estimation[2]:
            l += 1
        if pred['type_prediction'][3] == estimation[3]:
            l += 1
        n += 4
        
    
    print("---------------------------")
    print(f"MBTI 500 as Val Set\n")
    print(f"{j} fully correct MBTI types out of {i}\n")
    print(f"{round((100*(j/i)),5)}% correct full types\n")
    print(f"{l} MBTI letters correct out of {n} total\n")
    print(f"{round((100*(l/n)),5)}% correct individual types\n")


In [14]:
if FULL_PREDICT_TEST:
    # iterables
    j = 0
    l = 0
    n = 0
    # get data
    df = pd.read_csv('data/csv/mbti_1.csv')
    df.rename(columns={'posts': 'text'}, inplace=True)
    for i in range(len(df)):
        df_shuf = df.sample(n = 1, ignore_index=True)
        estimation = df_shuf['type'].tolist()[0].upper()
        df_pred = df_shuf.drop(columns='type')

        if BERT_LOAD or BERT_PREPROCESS:
            df_pred_list = bert_prediction_preprocessing(df_pred)
        else:
            df_pred_pp = prediction_preprocessing(df_pred)
            df_pred_list = prediction_vectorize(df_pred_pp)

        
        pred = predict_model(texts = df_pred_list, verbose = False)
        print(f"Prediction = {pred['type_prediction']} {pred['type_score']}")
        print(f"Actual = {estimation}")
        if pred['type_prediction'] == estimation:
            j += 1
        if pred['type_prediction'][0] == estimation[0]:
            l += 1
        if pred['type_prediction'][1] == estimation[1]:
            l += 1
        if pred['type_prediction'][2] == estimation[2]:
            l += 1
        if pred['type_prediction'][3] == estimation[3]:
            l += 1
        n += 4
            
    
    print("---------------------------")
    print(f"mbti_1 as Val Set\n")
    print(f"{j} fully correct MBTI types out of {i}\n")
    print(f"{round((100*(j/i)),5)}% correct full types\n")
    print(f"{l} MBTI letters correct out of {n} total\n")
    print(f"{round((100*(l/n)),5)}% correct individual types\n")